In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("Alle_Messungen_trimmed.csv")
df = df.drop(columns=["id", "user", "id_combined", "Unnamed: 0"])

# Convert time column to datetime and then to Unix timestamp
df['time'] = pd.to_datetime(df['time']).astype('int64')//1e9

# Encode the class labels
le = LabelEncoder()
df["class"] = le.fit_transform(df["class"])

# Set the number of time steps
n_steps = 5

# Get the input and output sequences
X = df.values[:, 1:13]
y = df.values[:, 13]

# Create sequences with sliding window
X_seq = []
for i in range(n_steps):
    X_seq.append(X[i:len(X)-n_steps+i+1])

for i in range(1, len(X)-n_steps+1):
    X_seq.append(X[i:i+n_steps])

# Split the sequences into equal-sized sequences of length n_steps
X_seq = np.array(X_seq)
X_seq = np.concatenate(X_seq, axis=1)
X_seq = np.split(X_seq, len(X_seq) / n_steps)
y_seq = y[n_steps-1:]

# Convert sequences to tensorflow dataset
dataset = tf.data.Dataset.from_tensor_slices((X_seq, y_seq))

# Split the dataset into train and test sets
train_size = int(0.7 * len(X_seq))
train_dataset = dataset.take(train_size).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = dataset.skip(train_size).batch(32).prefetch(tf.data.AUTOTUNE)

# Define the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(32, activation='relu', input_shape=(n_steps, 12)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=10, validation_data=test_dataset)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2559982,) + inhomogeneous part.